#### **Import Libraries**

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

#### **Data Cleaning**

**Import the Data**

In [40]:
trending_df = pd.read_csv('../data/US_youtube_trending_data.csv')


In [41]:
# load data using Python JSON module
with open('../data/US_category_id.json','r') as f:
    data = json.loads(f.read())
    
category_df = pd.json_normalize(data, record_path=['items'])

In [42]:
trending_df.head()

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11T19:20:14Z,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12T00:00:00Z,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156908,5855,35313,https://i.ytimg.com/vi/3C66w5Z0ixs/default.jpg,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...
1,M9Pmf9AB4Mo,Apex Legends | Stories from the Outlands – “Th...,2020-08-11T17:00:10Z,UC0ZV6M2THA81QT9hrVWJG3A,Apex Legends,20,2020-08-12T00:00:00Z,Apex Legends|Apex Legends characters|new Apex ...,2381688,146739,2794,16549,https://i.ytimg.com/vi/M9Pmf9AB4Mo/default.jpg,False,False,"While running her own modding shop, Ramya Pare..."
2,J78aPJ3VyNs,I left youtube for a month and THIS is what ha...,2020-08-11T16:34:06Z,UCYzPXprvl5Y-Sf0g4vX-m6g,jacksepticeye,24,2020-08-12T00:00:00Z,jacksepticeye|funny|funny meme|memes|jacksepti...,2038853,353787,2628,40221,https://i.ytimg.com/vi/J78aPJ3VyNs/default.jpg,False,False,I left youtube for a month and this is what ha...
3,kXLn3HkpjaA,XXL 2020 Freshman Class Revealed - Official An...,2020-08-11T16:38:55Z,UCbg_UMjlHJg_19SZckaKajg,XXL,10,2020-08-12T00:00:00Z,xxl freshman|xxl freshmen|2020 xxl freshman|20...,496771,23251,1856,7647,https://i.ytimg.com/vi/kXLn3HkpjaA/default.jpg,False,False,Subscribe to XXL → http://bit.ly/subscribe-xxl...
4,VIUo6yapDbc,Ultimate DIY Home Movie Theater for The LaBran...,2020-08-11T15:10:05Z,UCDVPcEbVLQgLZX0Rt6jo34A,Mr. Kate,26,2020-08-12T00:00:00Z,The LaBrant Family|DIY|Interior Design|Makeove...,1123889,45802,964,2196,https://i.ytimg.com/vi/VIUo6yapDbc/default.jpg,False,False,Transforming The LaBrant Family's empty white ...


In [43]:
trending_df.describe()

,categoryId,view_count,likes,dislikes,comment_count
count,177390.000000,1.773900e+05,1.773900e+05,177390.000000,1.773900e+05
mean,18.840267,2.531660e+06,1.328712e+05,1718.693528,1.115995e+04
std,6.751101,7.038882e+06,4.096811e+05,9854.973492,8.487794e+04
min,1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
25%,17.000000,4.850082e+05,1.920925e+04,0.000000,1.359000e+03
50%,20.000000,9.883235e+05,4.441650e+04,186.000000,3.034000e+03
75%,24.000000,2.225834e+06,1.111325e+05,1009.000000,7.214000e+03
max,29.000000,2.777917e+08,1.602153e+07,879354.000000,6.738537e+06


In [44]:
trending_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177390 entries, 0 to 177389
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   video_id           177390 non-null  object
 1   title              177390 non-null  object
 2   publishedAt        177390 non-null  object
 3   channelId          177390 non-null  object
 4   channelTitle       177390 non-null  object
 5   categoryId         177390 non-null  int64 
 6   trending_date      177390 non-null  object
 7   tags               177390 non-null  object
 8   view_count         177390 non-null  int64 
 9   likes              177390 non-null  int64 
 10  dislikes           177390 non-null  int64 
 11  comment_count      177390 non-null  int64 
 12  thumbnail_link     177390 non-null  object
 13  comments_disabled  177390 non-null  bool  
 14  ratings_disabled   177390 non-null  bool  
 15  description        173416 non-null  object
dtypes: bool(2), int64(5)

In [45]:
category_df.head()

,kind,etag,id,snippet.title,snippet.assignable,snippet.channelId
0,youtube#videoCategory,IfWa37JGcqZs-jZeAyFGkbeh6bc,1,Film & Animation,True,UCBR8-60-B28hp2BmDPdntcQ
1,youtube#videoCategory,5XGylIs7zkjHh5940dsT5862m1Y,2,Autos & Vehicles,True,UCBR8-60-B28hp2BmDPdntcQ
2,youtube#videoCategory,HCjFMARbBeWjpm6PDfReCOMOZGA,10,Music,True,UCBR8-60-B28hp2BmDPdntcQ
3,youtube#videoCategory,ra8H7xyAfmE2FewsDabE3TUSq10,15,Pets & Animals,True,UCBR8-60-B28hp2BmDPdntcQ
4,youtube#videoCategory,7mqChSJogdF3hSIL-88BfDE-W8M,17,Sports,True,UCBR8-60-B28hp2BmDPdntcQ


In [46]:
category_df.describe()

,kind,etag,id,snippet.title,snippet.assignable,snippet.channelId
count,32,32,32,32,32,32
unique,1,32,32,31,2,1
top,youtube#videoCategory,IfWa37JGcqZs-jZeAyFGkbeh6bc,1,Comedy,False,UCBR8-60-B28hp2BmDPdntcQ
freq,32,1,1,2,17,32


In [47]:
category_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   kind                32 non-null     object
 1   etag                32 non-null     object
 2   id                  32 non-null     object
 3   snippet.title       32 non-null     object
 4   snippet.assignable  32 non-null     bool  
 5   snippet.channelId   32 non-null     object
dtypes: bool(1), object(5)
memory usage: 1.4+ KB


In [48]:
# convert type of column to make merge successful
category_df['id'] = category_df['id'].astype('int64')

**Merge Datasets Into Single DataFrame**

In [49]:
df = pd.merge(trending_df, category_df, left_on=['channelId', 'categoryId'], right_on=['snippet.channelId', 'id'], how='inner')

In [50]:
df.head()

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,...,thumbnail_link,comments_disabled,ratings_disabled,description,kind,etag,id,snippet.title,snippet.assignable,snippet.channelId
0,gYPn3lbGRRc,Find a Way to learn #WithMe | Teachers helping...,2020-10-19T16:39:42Z,UCBR8-60-B28hp2BmDPdntcQ,YouTube,27,2020-10-26T00:00:00Z,Coronavirus|COVID|YouTube ad|YouTube advert|Wi...,4501889,8708,...,https://i.ytimg.com/vi/gYPn3lbGRRc/default.jpg,False,False,There was no lesson plan for how to teach in 2...,youtube#videoCategory,hHUhloYhyMMqVkQ4LH9sr3aRQoU,27,Education,True,UCBR8-60-B28hp2BmDPdntcQ
1,gYPn3lbGRRc,Find a Way to learn #WithMe | Teachers helping...,2020-10-19T16:39:42Z,UCBR8-60-B28hp2BmDPdntcQ,YouTube,27,2020-10-27T00:00:00Z,Coronavirus|COVID|YouTube ad|YouTube advert|Wi...,4668619,8729,...,https://i.ytimg.com/vi/gYPn3lbGRRc/default.jpg,False,False,There was no lesson plan for how to teach in 2...,youtube#videoCategory,hHUhloYhyMMqVkQ4LH9sr3aRQoU,27,Education,True,UCBR8-60-B28hp2BmDPdntcQ
2,h9bxHt7yDMo,Getting through 2020 | Ask How #WithMe,2020-12-15T17:42:27Z,UCBR8-60-B28hp2BmDPdntcQ,YouTube,27,2020-12-17T00:00:00Z,Coronavirus|COVID|YouTube ad|YouTube advert|Wi...,5139539,29854,...,https://i.ytimg.com/vi/h9bxHt7yDMo/default.jpg,False,False,"When 2020 left us all with questions, millions...",youtube#videoCategory,hHUhloYhyMMqVkQ4LH9sr3aRQoU,27,Education,True,UCBR8-60-B28hp2BmDPdntcQ
3,h9bxHt7yDMo,Getting through 2020 | Ask How #WithMe,2020-12-15T17:42:27Z,UCBR8-60-B28hp2BmDPdntcQ,YouTube,27,2020-12-18T00:00:00Z,Coronavirus|COVID|YouTube ad|YouTube advert|Wi...,5463371,30356,...,https://i.ytimg.com/vi/h9bxHt7yDMo/default.jpg,False,False,"When 2020 left us all with questions, millions...",youtube#videoCategory,hHUhloYhyMMqVkQ4LH9sr3aRQoU,27,Education,True,UCBR8-60-B28hp2BmDPdntcQ
4,h9bxHt7yDMo,Getting through 2020 | Ask How #WithMe,2020-12-15T17:42:27Z,UCBR8-60-B28hp2BmDPdntcQ,YouTube,27,2020-12-19T00:00:00Z,Coronavirus|COVID|YouTube ad|YouTube advert|Wi...,5723185,30626,...,https://i.ytimg.com/vi/h9bxHt7yDMo/default.jpg,False,False,"When 2020 left us all with questions, millions...",youtube#videoCategory,hHUhloYhyMMqVkQ4LH9sr3aRQoU,27,Education,True,UCBR8-60-B28hp2BmDPdntcQ


##### **Clean Merged Dataset**

In [51]:
category_col = df['snippet.title']
df = df.drop(labels=['snippet.channelId', 'id', 'snippet.assignable', 'etag', 'kind', 'categoryId', 'snippet.title', 'thumbnail_link'], axis=1)
df['category'] = category_col

In [52]:
df.head()

,video_id,title,publishedAt,channelId,channelTitle,trending_date,tags,view_count,likes,dislikes,comment_count,comments_disabled,ratings_disabled,description,category
0,gYPn3lbGRRc,Find a Way to learn #WithMe | Teachers helping...,2020-10-19T16:39:42Z,UCBR8-60-B28hp2BmDPdntcQ,YouTube,2020-10-26T00:00:00Z,Coronavirus|COVID|YouTube ad|YouTube advert|Wi...,4501889,8708,12073,2312,False,False,There was no lesson plan for how to teach in 2...,Education
1,gYPn3lbGRRc,Find a Way to learn #WithMe | Teachers helping...,2020-10-19T16:39:42Z,UCBR8-60-B28hp2BmDPdntcQ,YouTube,2020-10-27T00:00:00Z,Coronavirus|COVID|YouTube ad|YouTube advert|Wi...,4668619,8729,12145,2310,False,False,There was no lesson plan for how to teach in 2...,Education
2,h9bxHt7yDMo,Getting through 2020 | Ask How #WithMe,2020-12-15T17:42:27Z,UCBR8-60-B28hp2BmDPdntcQ,YouTube,2020-12-17T00:00:00Z,Coronavirus|COVID|YouTube ad|YouTube advert|Wi...,5139539,29854,37085,12607,False,False,"When 2020 left us all with questions, millions...",Education
3,h9bxHt7yDMo,Getting through 2020 | Ask How #WithMe,2020-12-15T17:42:27Z,UCBR8-60-B28hp2BmDPdntcQ,YouTube,2020-12-18T00:00:00Z,Coronavirus|COVID|YouTube ad|YouTube advert|Wi...,5463371,30356,37856,12801,False,False,"When 2020 left us all with questions, millions...",Education
4,h9bxHt7yDMo,Getting through 2020 | Ask How #WithMe,2020-12-15T17:42:27Z,UCBR8-60-B28hp2BmDPdntcQ,YouTube,2020-12-19T00:00:00Z,Coronavirus|COVID|YouTube ad|YouTube advert|Wi...,5723185,30626,38233,12998,False,False,"When 2020 left us all with questions, millions...",Education
